# Algo-Trading Starter (Google Colab)
Run algorithmic trading backtests and connect to Alpaca paper trading from **Android (via Colab)**.

In [ ]:
# Install dependencies
!pip install backtrader==1.9.78.123 pandas alpaca-trade-api python-dotenv matplotlib yfinance

In [ ]:
# Download example stock data (Apple 2018-2023)
import yfinance as yf
import os
os.makedirs('data', exist_ok=True)

data = yf.download("AAPL", start="2018-01-01", end="2023-12-31")
data.to_csv("data/example.csv")
print("Saved data/example.csv")

In [ ]:
# Backtest with backtrader
import backtrader as bt

class SimpleMeanReversion(bt.Strategy):
    params = dict(period=20, entry_z=-1.0, exit_z=0.0)

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.period)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=self.p.period)

    def next(self):
        z = (self.data.close[0] - self.sma[0]) / (self.std[0] + 1e-9)
        if not self.position:
            if z < self.p.entry_z:
                size = int(self.broker.getvalue() * 0.02 / self.data.close[0])
                if size > 0:
                    self.buy(size=size)
        else:
            if z > self.p.exit_z:
                self.close()

cerebro = bt.Cerebro()
cerebro.addstrategy(SimpleMeanReversion)
data = bt.feeds.YahooFinanceCSVData(dataname='data/example.csv')
cerebro.adddata(data)
cerebro.broker.setcash(100000)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot()

In [ ]:
# Alpaca Paper Trading Connection
import os
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST

# Save API keys in .env file (edit here)
with open(".env", "w") as f:
    f.write("APCA_API_KEY_ID=your_key_here\n")
    f.write("APCA_API_SECRET_KEY=your_secret_here\n")

load_dotenv(".env")
API_KEY = os.getenv('APCA_API_KEY_ID')
API_SECRET = os.getenv('APCA_API_SECRET_KEY')
BASE_URL = 'https://paper-api.alpaca.markets'

api = REST(API_KEY, API_SECRET, BASE_URL)
account = api.get_account()
print('Account status:', account.status)
print('Cash:', account.cash)

# Example order (paper only, uncomment to test)
# order = api.submit_order(symbol='AAPL', qty=1, side='buy', type='market', time_in_force='gtc')
# print(order)

### ✅ Next steps
- Run backtests on different symbols.
- Keep Alpaca trading in **paper mode** until stable.
- Never share API keys.
- Add risk controls before going live.